<!-- PROJECT LOGO -->
<br/>
<p align="center">
    <img src="https://raw.githubusercontent.com/Team-17-Bedu/proyecto_python/main/img/icono.png" alt="Logo" width="135" height="135">

  <h3 align="center"><strong>Países con mayor calidad de vida</strong></h3>

  <p align="center">
    Proyecto final para el modulo "Procesamiento de datos con Python"
  </p>
</p>

#### Acerca del proyecto:

Este proyecto tiene como proposito realizar un analisis de la calidad de vida de ciertos paises, con el fin de determinar cuál de ellos podria ser la mejor opcion para vivir o buscar un empleo, todo esto, haciendo uso de herramientas estadísticas y computacionales, siendo el principal aliado, el lenguaje de programacion Python. 


En este caso se tendra como principal objetivo en calcular el IDH por medio de la documentación proporcionada del __*Human Development Report 2020*__, presente en el siguiente [enlace](http://hdr.undp.org/sites/default/files/hdr2020_technical_notes.pdf). Y con tales resultados justificar nuestro anterior proyecto elaborado en el lenguaje R, el proyecto esta presente en el siguiente [enlace](https://github.com/Team-17-Bedu/proyecto).

Todos los indicadores serán obtenidos por medio de la API proporcionada por la UNDP, de los cuales serán analizados 
* Años esperados de escolaridad 
* Ingreso nacional bruto (INB) per cápita
* Esperanza de vida al nacer
* Promedio de años de escolaridad

#### Preparación:
Para comenzar se importaron los modulos necesarios (dadas las limitaciones de github no es posible renderizar los widgets, para ello sera necesario visualuzar el notebook en Google Colab)


In [1]:
import pandas as pd
import requests
import ipywidgets as widgets
from IPython.display import display

Debido a que el acceso a la API requiere un inicio de sesion, se recurrió al objeto `Session` del modulo `requests`  

In [2]:
session = requests.Session()

Mediante el modulo `widgets` se definieron dos text box para introducir los datos de autenticación, `user` y `password` respectivamente 

In [3]:
user = widgets.Text()
password = widgets.Text()
display(user, password)

Text(value='')

Text(value='')

Definición de la tupla que contendra los datos de acceso

In [4]:
access = (user.value, password.value)

Inicio de sesión en la API de la UNDP

In [5]:
session.get("http://ec2-54-174-131-205.compute-1.amazonaws.com/API/Login.php",
            auth=access)

<Response [200]>

Dada la estructura del `json` recibido, se construyo una funcion para normalizar los datos, a modo de obtener un `DataFrame` homogeneo

In [6]:
def normalize_data(data_dirt: dict, indicator_id: str) -> dict:
    new_data = {
        'Country': list(data_dirt['country_name'].values())
    }
    countries = list(data_dirt['country_name'].keys())
    for year, year_data in data_dirt['indicator_value'].items():
        values = []
        last_key = ''
        for key, val in year_data[indicator_id].items():
            current_pos = countries.index(key)
            last_pos = countries.index(last_key) if last_key != '' else 0
            last_key = key
            zeros = current_pos - last_pos
            i = 1
            while i < zeros and zeros != 0:
                values.append(0)
                i += 1
            values.append(val)
        new_data[year] = values
    return  new_data

### Calculo del Indice de Educación
#### Expectativa de años de escolaridad
Definición del indicador a consultar, en este caso el correspondiente a la expectativa de años de escolaridad

In [7]:
indicator = '69706'

Realización de la consulta (puesto que la estructura es diferente a la de otras peticiones, esta se realizo con un `fstring`)

In [8]:
data = session.get(f'http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/indicator_id={indicator}/structure=yic')
data

<Response [200]>

Normalizacion del `json` obtenido en la consulta

In [9]:
data_clean = normalize_data(data.json(), indicator)

Creación del `DataFrame` con los datos normalizados y visualización de `head` y `tail`

In [10]:
expectativa = pd.DataFrame(data_clean)
expectativa

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,2.593,2.919,3.246,3.573,3.899,4.226,4.553,4.879,5.206,...,9.532,9.478,9.973,10.172,10.262,10.235,10.262,10.139,10.139,10.176
1,Angola,3.441,3.252,3.234,3.657,3.767,3.877,3.987,4.096,4.206,...,8.646,9.545,9.916,10.286,10.657,11.028,11.399,11.777,11.777,11.777
2,Albania,11.603,11.764,10.664,10.127,10.091,10.166,10.228,10.505,10.669,...,13.000,13.748,14.587,14.926,15.252,15.076,14.805,14.816,14.696,14.696
3,Andorra,10.799,10.799,10.799,10.799,10.799,10.799,10.799,10.799,10.799,...,11.672,11.672,13.524,13.139,13.495,13.140,13.300,13.046,13.300,13.300
4,United Arab Emirates,10.314,10.663,10.507,10.539,10.860,11.093,10.847,10.550,10.376,...,12.208,12.361,12.514,12.666,13.005,13.667,13.643,13.643,14.344,14.344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Samoa,11.655,11.699,11.744,11.788,11.832,11.877,11.921,11.832,12.010,...,12.859,12.508,12.380,12.458,12.535,12.433,12.431,12.520,12.520,12.730
189,Yemen,7.531,7.545,7.559,7.573,7.587,7.602,7.616,7.630,7.594,...,8.589,8.977,8.457,8.809,8.737,8.701,8.664,8.664,8.664,8.769
190,South Africa,11.371,11.946,12.315,12.684,12.752,13.038,13.022,13.007,12.992,...,12.808,12.793,12.872,13.147,13.406,13.765,13.668,13.668,13.668,13.791
191,Zambia,7.518,7.744,7.971,8.197,8.424,8.650,8.876,9.103,9.329,...,11.017,10.932,10.931,10.960,11.046,11.133,11.219,11.305,11.392,11.478


* Dimensiones del `DataFrame`
    * 193 filas
    * 31 columnas

In [11]:
expectativa.shape

(193, 31)

* Nombre de las columnas

In [12]:
expectativa.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

#### Calculo del indice de expectativa de años de escolaridad por paises

Se explica a continuación como se realizo el calculo del indice de expectativa de años de escolaaridad por pais.
Primeramente se presenta lo que es la formula que se uso:

$$\Huge\Huge\frac{\alpha - \theta}{\gamma - \theta}$$

Donde:

* $\alpha$ : La expectativa de escolaridad en el pais.
* $\theta$ : Es la expectativa minima de años de escolaridad. En el documento __*Human Development Report 2020*__ se usa 0 años como minimo.
* $\gamma$ : Es la expectativa maxima de años de escolaridad. En el documento __*Human Development Report 2020*__ se usa 18 años como maximo.


Este calculo se realizo por cada pais en cada uno de los años.

In [13]:
for year in range(1990, 2020):
    year = str(year)
    indices = []
    for cell in expectativa[year]:
        try:
            indices.append((float(cell) - 0) / (18 - 0))
        except:
            indices.append(0)
    expectativa[f'Indice_Expect_Educacion_{year}'] = indices

expectativa = expectativa.drop([str(i) for i in range(1990, 2020)],axis=1)

In [14]:
expectativa.head()

,Country,Indice_Expect_Educacion_1990,Indice_Expect_Educacion_1991,Indice_Expect_Educacion_1992,Indice_Expect_Educacion_1993,Indice_Expect_Educacion_1994,Indice_Expect_Educacion_1995,Indice_Expect_Educacion_1996,Indice_Expect_Educacion_1997,Indice_Expect_Educacion_1998,...,Indice_Expect_Educacion_2010,Indice_Expect_Educacion_2011,Indice_Expect_Educacion_2012,Indice_Expect_Educacion_2013,Indice_Expect_Educacion_2014,Indice_Expect_Educacion_2015,Indice_Expect_Educacion_2016,Indice_Expect_Educacion_2017,Indice_Expect_Educacion_2018,Indice_Expect_Educacion_2019
0,Afghanistan,0.144056,0.162167,0.180333,0.198500,0.216611,0.234778,0.252944,0.271056,0.289222,...,0.529556,0.526556,0.554056,0.565111,0.570111,0.568611,0.570111,0.563278,0.563278,0.565333
1,Angola,0.191167,0.180667,0.179667,0.203167,0.209278,0.215389,0.221500,0.227556,0.233667,...,0.480333,0.530278,0.550889,0.571444,0.592056,0.612667,0.633278,0.654278,0.654278,0.654278
2,Albania,0.644611,0.653556,0.592444,0.562611,0.560611,0.564778,0.568222,0.583611,0.592722,...,0.722222,0.763778,0.810389,0.829222,0.847333,0.837556,0.822500,0.823111,0.816444,0.816444
3,Andorra,0.599944,0.599944,0.599944,0.599944,0.599944,0.599944,0.599944,0.599944,0.599944,...,0.648444,0.648444,0.751333,0.729944,0.749722,0.730000,0.738889,0.724778,0.738889,0.738889
4,United Arab Emirates,0.573000,0.592389,0.583722,0.585500,0.603333,0.616278,0.602611,0.586111,0.576444,...,0.678222,0.686722,0.695222,0.703667,0.722500,0.759278,0.757944,0.757944,0.796889,0.796889


#### Expectativa de años de escolaridad
Definición del indicador a consultar, en este caso el correspondiente al promedio de años de escolaridad

In [15]:
indicator = '103006'

Realización de la consulta (puesto que la estructura es diferente a la de otras peticiones, esta se realizo con un `fstring`)

In [16]:
data = session.get(f'http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/indicator_id={indicator}/structure=yic')
data

<Response [200]>

Normalizacion del `json` obtenido en la consulta

In [17]:
data_clean = normalize_data(data.json(), indicator)

Creación del `DataFrame` con los datos normalizados y visualización de `head` y `tail`

In [18]:
promedio = pd.DataFrame(data_clean)
promedio

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,1.490,1.564,1.638,1.712,1.786,1.860,1.920,1.980,2.040,...,3.230,3.310,3.390,3.470,3.550,3.630,3.630,3.780,3.930,3.930
1,Angola,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,4.726,4.726,4.788,4.851,4.915,4.980,5.100,5.125,5.174,5.174
2,Albania,7.830,7.828,7.826,7.824,7.822,8.027,8.175,8.323,8.471,...,9.292,9.958,10.025,10.025,10.025,10.025,10.025,10.055,10.055,10.146
3,Andorra,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,10.364,10.402,10.439,10.477,10.515,10.590,10.590,10.519,10.502,10.502
4,United Arab Emirates,5.623,5.915,6.207,6.500,6.792,7.084,7.323,7.562,7.802,...,9.883,10.036,10.189,10.342,10.495,10.648,10.923,12.111,12.111,12.111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,Samoa,7.602,7.724,7.845,7.966,8.088,8.209,8.331,8.452,8.573,...,10.030,10.490,10.511,10.532,10.553,10.574,10.595,10.595,10.595,10.779
186,Yemen,0.290,0.362,0.434,0.506,0.578,0.650,0.760,0.870,0.980,...,2.600,2.800,3.000,3.000,3.000,3.000,3.000,3.000,3.200,3.200
187,South Africa,6.490,6.600,7.177,7.523,7.869,8.215,8.327,8.438,8.549,...,10.215,9.574,9.821,9.905,9.989,10.134,10.155,10.155,10.241,10.241
188,Zambia,4.680,4.952,5.224,5.496,5.768,6.040,6.008,5.976,5.944,...,6.600,6.674,6.748,6.822,6.769,6.864,6.960,7.056,7.104,7.152


* Dimensiones del `DataFrame`
    * 190 filas
    * 31 columnas

In [19]:
promedio.shape

(190, 31)

* Nombre de las columnas

In [20]:
promedio.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

#### Calculo del indice de promedio de años de escolaridad por país

Se explica a continuación como se realizo el calculo del indice de promedio de años de escolaridad por pais.
Primeramente se presenta lo que es la formula que se uso:

$$\Huge\Huge\frac{\alpha - \theta}{\gamma - \theta}$$

Donde:

* $\alpha$ : El promedio de años de escolaridad en el pais.
* $\theta$ : Es la expectativa minima del promedio de años de escolaridad. En el documento __*Human Development Report 2020*__ se usa 0 años como minimo.
* $\gamma$ : Es la expectativa maxima del promedio de años de escolaridad. En el documento __*Human Development Report 2020*__ se usa 15 años como maximo.


Este calculo se realizo por cada pais en cada uno de los años. 

In [21]:
for year in range(1990, 2020):
    year = str(year)
    indices = []
    for cell in promedio[year]:
        try:
            indices.append((float(cell) - 0) / (15 - 0))
        except:
            indices.append(0)
    promedio[f'Indice_Promedio_Edu_{year}'] = indices

promedio = promedio.drop([str(i) for i in range(1990, 2020)], axis=1)

Resultado del calculo del indice del promedio de años de escolaridad por cada país en cada uno de sus periodos.

In [22]:
promedio.head()

,Country,Indice_Promedio_Edu_1990,Indice_Promedio_Edu_1991,Indice_Promedio_Edu_1992,Indice_Promedio_Edu_1993,Indice_Promedio_Edu_1994,Indice_Promedio_Edu_1995,Indice_Promedio_Edu_1996,Indice_Promedio_Edu_1997,Indice_Promedio_Edu_1998,...,Indice_Promedio_Edu_2010,Indice_Promedio_Edu_2011,Indice_Promedio_Edu_2012,Indice_Promedio_Edu_2013,Indice_Promedio_Edu_2014,Indice_Promedio_Edu_2015,Indice_Promedio_Edu_2016,Indice_Promedio_Edu_2017,Indice_Promedio_Edu_2018,Indice_Promedio_Edu_2019
0,Afghanistan,0.099333,0.104267,0.109200,0.114133,0.119067,0.124000,0.1280,0.132000,0.136000,...,0.215333,0.220667,0.226000,0.231333,0.236667,0.242000,0.242000,0.252000,0.262000,0.262000
1,Angola,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.315067,0.315067,0.319200,0.323400,0.327667,0.332000,0.340000,0.341667,0.344933,0.344933
2,Albania,0.522000,0.521867,0.521733,0.521600,0.521467,0.535133,0.5450,0.554867,0.564733,...,0.619467,0.663867,0.668333,0.668333,0.668333,0.668333,0.668333,0.670333,0.670333,0.676400
3,Andorra,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.690933,0.693467,0.695933,0.698467,0.701000,0.706000,0.706000,0.701267,0.700133,0.700133
4,United Arab Emirates,0.374867,0.394333,0.413800,0.433333,0.452800,0.472267,0.4882,0.504133,0.520133,...,0.658867,0.669067,0.679267,0.689467,0.699667,0.709867,0.728200,0.807400,0.807400,0.807400


#### Calculo del Indice de Educacion
Creación de un `DataFrame` vacío para almacenar todos los resultados del calculo 

In [23]:
header = ['Pais']
indices = pd.DataFrame({}, columns=header)

### Calculo del indice de Educación por país

Se explica a continuación como se realizo el calculo del indice de educación por país.
Primeramente se presenta lo que es la formula que se uso:

$$\Huge\Huge\frac{\alpha + \beta}{2}$$

Donde:

* $\alpha$ : El indice de la expectativa de años de escolaridad en el pais.
* $\beta$ : El indice del promedio de años de escolaridad en el país

Este calculo se realizo por cada pais en cada uno de los años. 


In [24]:
def calcular_indice(_pais, expect, prom):
    dicci = {
        "Pais": _pais
    }
    prom, expect = prom[0][1:], expect[0][1:]
    indice = [(float(prom[i]) + float(expect[i])) / 2 for i in range(0, len(prom))]

    for i in range(1990, 2020):
        dicci[str(i)] = indice[i - 1990]
    return dicci


for country in promedio["Country"]:
    expect = expectativa.loc[expectativa.Country == country].to_numpy()
    prom = promedio.loc[promedio.Country == country].to_numpy()
    if len(expect) > 0 and len(prom) > 0:
        indices = indices.append(calcular_indice(country, expect, prom), ignore_index= True)


Resultado del calculo del indice de educción por cada país en cada uno de sus periodos.

In [25]:
indices.head()

,Pais,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.121694,0.133217,0.144767,0.156317,0.167839,0.179389,0.190472,0.201528,0.212611,...,0.372444,0.373611,0.390028,0.398222,0.403389,0.405306,0.406056,0.407639,0.412639,0.413667
1,Angola,0.095583,0.090333,0.089833,0.101583,0.104639,0.107694,0.110750,0.113778,0.116833,...,0.397700,0.422672,0.435044,0.447422,0.459861,0.472333,0.486639,0.497972,0.499606,0.499606
2,Albania,0.583306,0.587711,0.557089,0.542106,0.541039,0.549956,0.556611,0.569239,0.578728,...,0.670844,0.713822,0.739361,0.748778,0.757833,0.752944,0.745417,0.746722,0.743389,0.746422
3,Andorra,0.299972,0.299972,0.299972,0.299972,0.299972,0.299972,0.299972,0.299972,0.299972,...,0.669689,0.670956,0.723633,0.714206,0.725361,0.718000,0.722444,0.713022,0.719511,0.719511
4,United Arab Emirates,0.473933,0.493361,0.498761,0.509417,0.528067,0.544272,0.545406,0.545122,0.548289,...,0.668544,0.677894,0.687244,0.696567,0.711083,0.734572,0.743072,0.782672,0.802144,0.802144
